# Load dataset

## Import the pandas library and perform data reading for analysis.

In [1]:
import pandas as pd
train = pd.read_csv('train.csv', sep='\t', header=None)
test = pd.read_csv('test.csv', sep='\t', header=None)

## Adding column names to data.

In [2]:
train.columns = ["text",'labels']
test.columns = ["text",'labels']

## The first 10 texts of the training set are read in the format "text, labels". The labels may contain multiple sentiment categories, each sentiment is separated by ','.

In [3]:
train.head(10)

,text,labels
0,My favourite food is anything I didn't have to...,27
1,"Now if he does off himself, everyone will thin...",27
2,WHY THE FUCK IS BAYLESS ISOING,2
3,To make her feel threatened,14
4,Dirty Southern Wankers,3
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26
6,Yes I heard abt the f bombs! That has to be wh...,15
7,We need more boards and to create a bit more s...,"8,20"
8,Damn youtube and outrage drama is super lucrat...,0
9,It might be linked to the trust factor of your...,27


In [4]:
test.head(10)

,text,labels
0,I’m really sorry about your situation :( Altho...,25
1,It's wonderful because it's awful. At not with.,0
2,"Kings fan here, good luck to you guys! Will be...",13
3,"I didn't know that, thank you for teaching me ...",15
4,They got bored from haunting earth for thousan...,27
5,Thank you for asking questions and recognizing...,15
6,You’re welcome,15
7,100%! Congrats on your job too!,15
8,I’m sorry to hear that friend :(. It’s for the...,24
9,"Girlfriend weak as well, that jump was pathetic.",25


# Import related libraries.

In [5]:
import os
import paddle
import paddlenlp

/Users/wubowei/miniforge3/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/wubowei/miniforge3/envs/nlp/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Data pre-processing

## For the 28 micro-sentiment multi-label classification scenario, where a sentence may correspond to multiple sentiment category labels, the sentiment labels of the dataset need to be transformed using One-Hot coding first, with "0" indicating absence and "1" indicating presence for each sentiment.

### Create sentiment label mapping relationships.

In [6]:
label_vocab = {
    0: "admiration",
    1: "amusement",
    2: "anger",
    3: "annoyance",
    4: "approval",
    5: "caring",
    6: "confusion",
    7: "curiosity",
    8: "desire",
    9: "disappointment",
    10: "disapproval",
    11: "disgust",
    12: "embarrassment",
    13: "excitement",
    14: "fear",
    15: "gratitude",
    16: "grief",
    17: "joy",
    18: "love",
    19: "nervousness",
    20: "optimism",
    21: "pride",
    22: "realization",
    23: "relief",
    24: "remorse",
    25: "sadness",
    26: "surprise",
    27: "neutral"
}

### Customize the dataset, read the data file, create the dataset and define the data type as MapDataset.

In [7]:
import re

from paddlenlp.datasets import load_dataset

# Clear invalid characters
def clean_text(text):
    text = text.replace("\r", "").replace("\n", "")
    text = re.sub(r"\\n\n", ".", text)
    return text

# Define the read data set function
def read_custom_data(filepath, is_one_hot=True):
    f = open(filepath)
    while True:
        line = f.readline()
        if not line:
            break
        data = line.strip().split('\t')
        # One-hot processing for 28 types of micro sentiment tags
        if is_one_hot:
            labels = [float(1) if str(i) in data[1].split(',') else float(0) for i in range(28)]  # 28 types
        else:
            labels = [int(d) for d in data[1].split(',')]
        yield {"text": clean_text(data[0]), "labels": labels}
    f.close()

In [8]:
# load_dataset() to Create dataset.
# lazy=False，The dataset is returned as a MapDataset type.
# Pre-processing of training and validation sets.
train_ds = load_dataset(read_custom_data, filepath='train.csv', lazy=False) 
test_ds = load_dataset(read_custom_data, filepath='test.csv', lazy=False)

### Print dataset.

In [9]:
print("datatype:", type(train_ds))
print("training dataset example:", train_ds[0])
print("testing dataset example:", test_ds[0])

datatype: <class 'paddlenlp.datasets.dataset.MapDataset'>
training dataset example: {'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]}
testing dataset example: {'text': 'I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!', 'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]}


## Load Chinese ERNIE 3.0 pre-training model and word splitter

In [10]:
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "ernie-3.0-medium-zh"   # ERNIE3.0 model
num_classes = 28  # 28 classification mission
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_classes=num_classes)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2023-04-13 18:50:10,193] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-3.0-medium-zh'.
[2023-04-13 18:50:10,194] [    INFO] - Model config ErnieConfig {
  "attention_probs_dropout_prob": 0.1,
  "enable_recompute": false,
  "fuse": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27"


## Process the raw data into a model-acceptable format.

In [11]:
import functools
import numpy as np

from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorWithPadding

# Data pre-processing function to convert text into integer sequences using a word splitter.
def preprocess_function(examples, tokenizer, max_seq_length):
    result = tokenizer(text=examples["text"], max_seq_len=max_seq_length)
    result["labels"] = examples["labels"]
    return result

trans_func = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=64)
train_ds = train_ds.map(trans_func)
test_ds = test_ds.map(trans_func)

# function is constructed to extend the different length sequences to the maximum length of the data in the batch, and then stack the data.
collate_fn = DataCollatorWithPadding(tokenizer)

# Define the BatchSampler, select the batch size and whether to randomly jumble the DataLoader.
train_batch_sampler = BatchSampler(train_ds, batch_size=32, shuffle=True)
test_batch_sampler = BatchSampler(test_ds, batch_size=16, shuffle=False)
train_data_loader = DataLoader(dataset=train_ds, batch_sampler=train_batch_sampler, collate_fn=collate_fn)
test_data_loader = DataLoader(dataset=test_ds, batch_sampler=test_batch_sampler, collate_fn=collate_fn)

## Define model validation metrics.

In [12]:
import numpy as np
import sklearn
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from paddle.metric import Metric

# Customize MultiLabelReport evaluation metrics.
class MultiLabelReport(Metric):
    """
    AUC and F1 Score for multi-label text classification task.
    """

    def __init__(self, name='MultiLabelReport', average='micro'):
        super(MultiLabelReport, self).__init__()
        self.average = average
        self._name = name
        self.reset()

    def f1_score(self, y_prob):
        '''
        Returns the f1 score by searching the best threshhold
        '''
        best_score = 0
        for threshold in [i * 0.01 for i in range(100)]:
            self.y_pred = y_prob > threshold
            score = sklearn.metrics.f1_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
            if score > best_score:
                best_score = score
                precison = precision_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
                recall = recall_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
        return best_score, precison, recall

    def reset(self):
        """
        Resets all of the metric state.
        """
        self.y_prob = None
        self.y_true = None

    def update(self, probs, labels):
        if self.y_prob is not None:
            self.y_prob = np.append(self.y_prob, probs.numpy(), axis=0)
        else:
            self.y_prob = probs.numpy()
        if self.y_true is not None:
            self.y_true = np.append(self.y_true, labels.numpy(), axis=0)
        else:
            self.y_true = labels.numpy()

    def accumulate(self):
        auc = roc_auc_score(
            y_score=self.y_prob, y_true=self.y_true, average=self.average)
        f1_score, precison, recall = self.f1_score(y_prob=self.y_prob)
        return auc, f1_score, precison, recall

    def name(self):
        """
        Returns metric name
        """
        return self._name

# Building the training model.

## Select an optimization strategy and run configuration.

In [13]:
import time
import paddle.nn.functional as F

# AdamW optimizer, cross-entropy loss function, custom MultiLabelReport evaluation metrics.
optimizer = paddle.optimizer.Adam(learning_rate=4e-4, parameters=model.parameters(), weight_decay=0.01)
criterion = paddle.nn.BCEWithLogitsLoss()
metric = MultiLabelReport()

## Model training and validation.

In [14]:
import paddle
import numpy as np
import paddle.nn.functional as F

# Build the validation set evaluate function.
@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader, label_vocab, if_return_results=True):
    model.eval()
    metric.reset()
    losses = []
    results = []
    for batch in data_loader:
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        probs = F.sigmoid(logits)
        losses.append(loss.numpy())
        metric.update(probs, labels)
        if if_return_results:
            probs = probs.tolist()
            for prob in probs:
                result = []
                for c, pred in enumerate(prob):
                    if pred > 0.5:
                        result.append(label_vocab[c])
                results.append(','.join(result))

    auc, f1_score, precison, recall = metric.accumulate()
    print("eval loss: %.5f, auc: %.5f, f1 score: %.5f, precison: %.5f, recall: %.5f" %
          (np.mean(losses), auc, f1_score, precison, recall))
    model.train()
    metric.reset()
    if if_return_results:
        return results
    else:
        return f1_score

In [15]:
epochs = 3 # training times
ckpt_dir = "ernie_ckpt" # Folder for saving model parameters during training

global_step = 0  # Number of iterations
tic_train = time.time()
best_f1_score = 0

# Model Training
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # Calculate model output, loss function value, classification probability value, accuracy, f1 score.
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        probs = F.sigmoid(logits)
        metric.update(probs, labels)
        auc, f1_score, _, _ = metric.accumulate()

        # Print the loss function value, accuracy, f1 score, and computation speed for each 10 iterations.
        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, auc: %.5f, f1 score: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, auc, f1_score,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # Reverse gradient passback with updated parameters.
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        
        # Every 40 iterations, evaluate the current trained model, save the current best model parameters and word list of the word splitter, etc.
        if global_step % 40 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            eval_f1_score = evaluate(model, criterion, metric, test_data_loader, label_vocab, if_return_results=False)
            if eval_f1_score > best_f1_score:
                best_f1_score = eval_f1_score
                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)

global step 10, epoch: 1, batch: 10, loss: 0.17966, auc: 0.58541, f1 score: 0.12083, speed: 0.25 step/s
global step 20, epoch: 1, batch: 20, loss: 0.14193, auc: 0.63329, f1 score: 0.13236, speed: 0.23 step/s
global step 30, epoch: 1, batch: 30, loss: 0.13396, auc: 0.67223, f1 score: 0.15816, speed: 0.24 step/s
global step 40, epoch: 1, batch: 40, loss: 0.13795, auc: 0.68744, f1 score: 0.17741, speed: 0.23 step/s


[2023-04-13 19:03:13,222] [    INFO] - Configuration saved in ernie_ckpt/config.json


eval loss: 0.14903, auc: 0.76551, f1 score: 0.30401, precison: 0.32928, recall: 0.28235


[2023-04-13 19:03:13,455] [    INFO] - tokenizer config file saved in ernie_ckpt/tokenizer_config.json
[2023-04-13 19:03:13,456] [    INFO] - Special tokens file saved in ernie_ckpt/special_tokens_map.json


global step 50, epoch: 1, batch: 50, loss: 0.13824, auc: 0.76624, f1 score: 0.31503, speed: 0.02 step/s
global step 60, epoch: 1, batch: 60, loss: 0.15719, auc: 0.75002, f1 score: 0.30145, speed: 0.24 step/s
global step 70, epoch: 1, batch: 70, loss: 0.13346, auc: 0.75585, f1 score: 0.30813, speed: 0.25 step/s
global step 80, epoch: 1, batch: 80, loss: 0.15162, auc: 0.75809, f1 score: 0.31223, speed: 0.22 step/s
eval loss: 0.14933, auc: 0.75965, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 90, epoch: 1, batch: 90, loss: 0.15405, auc: 0.75153, f1 score: 0.29898, speed: 0.04 step/s
global step 100, epoch: 1, batch: 100, loss: 0.14373, auc: 0.75497, f1 score: 0.29275, speed: 0.23 step/s
global step 110, epoch: 1, batch: 110, loss: 0.12937, auc: 0.76326, f1 score: 0.29597, speed: 0.24 step/s
global step 120, epoch: 1, batch: 120, loss: 0.11750, auc: 0.76260, f1 score: 0.30529, speed: 0.23 step/s
eval loss: 0.14865, auc: 0.76221, f1 score: 0.30401, precison: 0.32928, re

...

global step 1540, epoch: 2, batch: 13, loss: 0.13412, auc: 0.76218, f1 score: 0.31405, speed: 0.24 step/s
global step 1550, epoch: 2, batch: 23, loss: 0.14905, auc: 0.76189, f1 score: 0.29528, speed: 0.23 step/s
global step 1560, epoch: 2, batch: 33, loss: 0.13116, auc: 0.75941, f1 score: 0.29850, speed: 0.22 step/s
eval loss: 0.14907, auc: 0.76003, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 1570, epoch: 2, batch: 43, loss: 0.14438, auc: 0.74923, f1 score: 0.26973, speed: 0.04 step/s
global step 1580, epoch: 2, batch: 53, loss: 0.14820, auc: 0.75724, f1 score: 0.28025, speed: 0.24 step/s
global step 1590, epoch: 2, batch: 63, loss: 0.16206, auc: 0.76256, f1 score: 0.29375, speed: 0.23 step/s
global step 1600, epoch: 2, batch: 73, loss: 0.15541, auc: 0.76276, f1 score: 0.29033, speed: 0.24 step/s
eval loss: 0.14866, auc: 0.76182, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 1610, epoch: 2, batch: 83, loss: 0.17245, auc: 0.75709, f1 score: 0.30

global step 2180, epoch: 2, batch: 653, loss: 0.14126, auc: 0.75394, f1 score: 0.27023, speed: 0.25 step/s
global step 2190, epoch: 2, batch: 663, loss: 0.14970, auc: 0.75778, f1 score: 0.28104, speed: 0.25 step/s
global step 2200, epoch: 2, batch: 673, loss: 0.15275, auc: 0.75949, f1 score: 0.28056, speed: 0.24 step/s
eval loss: 0.14931, auc: 0.76006, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 2210, epoch: 2, batch: 683, loss: 0.12675, auc: 0.76982, f1 score: 0.36752, speed: 0.04 step/s
global step 2220, epoch: 2, batch: 693, loss: 0.17111, auc: 0.75379, f1 score: 0.32166, speed: 0.24 step/s
global step 2230, epoch: 2, batch: 703, loss: 0.14575, auc: 0.75554, f1 score: 0.32443, speed: 0.24 step/s
global step 2240, epoch: 2, batch: 713, loss: 0.15150, auc: 0.75814, f1 score: 0.32487, speed: 0.22 step/s
eval loss: 0.14922, auc: 0.76198, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 2250, epoch: 2, batch: 723, loss: 0.18107, auc: 0.76444, f1 sco

global step 2820, epoch: 2, batch: 1293, loss: 0.14262, auc: 0.74839, f1 score: 0.29979, speed: 0.24 step/s
global step 2830, epoch: 2, batch: 1303, loss: 0.15926, auc: 0.74842, f1 score: 0.29264, speed: 0.23 step/s
global step 2840, epoch: 2, batch: 1313, loss: 0.14454, auc: 0.75273, f1 score: 0.29112, speed: 0.24 step/s
eval loss: 0.14997, auc: 0.76183, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 2850, epoch: 2, batch: 1323, loss: 0.14199, auc: 0.76461, f1 score: 0.33091, speed: 0.04 step/s
global step 2860, epoch: 2, batch: 1333, loss: 0.14769, auc: 0.75624, f1 score: 0.32272, speed: 0.24 step/s
global step 2870, epoch: 2, batch: 1343, loss: 0.14802, auc: 0.75302, f1 score: 0.30466, speed: 0.24 step/s
global step 2880, epoch: 2, batch: 1353, loss: 0.15119, auc: 0.75841, f1 score: 0.30664, speed: 0.23 step/s
eval loss: 0.14857, auc: 0.76560, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 2890, epoch: 2, batch: 1363, loss: 0.15793, auc: 0.76318

global step 3460, epoch: 3, batch: 406, loss: 0.14254, auc: 0.75368, f1 score: 0.30255, speed: 0.24 step/s
global step 3470, epoch: 3, batch: 416, loss: 0.15411, auc: 0.75441, f1 score: 0.30145, speed: 0.24 step/s
global step 3480, epoch: 3, batch: 426, loss: 0.16395, auc: 0.75210, f1 score: 0.28963, speed: 0.23 step/s
eval loss: 0.14994, auc: 0.76046, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 3490, epoch: 3, batch: 436, loss: 0.14546, auc: 0.76865, f1 score: 0.28369, speed: 0.04 step/s
global step 3500, epoch: 3, batch: 446, loss: 0.14554, auc: 0.76677, f1 score: 0.27874, speed: 0.23 step/s
global step 3510, epoch: 3, batch: 456, loss: 0.16816, auc: 0.75890, f1 score: 0.28256, speed: 0.23 step/s
global step 3520, epoch: 3, batch: 466, loss: 0.14647, auc: 0.75780, f1 score: 0.28859, speed: 0.24 step/s
eval loss: 0.14977, auc: 0.75555, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 3530, epoch: 3, batch: 476, loss: 0.15646, auc: 0.74416, f1 sco

global step 4100, epoch: 3, batch: 1046, loss: 0.14522, auc: 0.75982, f1 score: 0.31709, speed: 0.24 step/s
global step 4110, epoch: 3, batch: 1056, loss: 0.14642, auc: 0.75465, f1 score: 0.30754, speed: 0.24 step/s
global step 4120, epoch: 3, batch: 1066, loss: 0.14288, auc: 0.76195, f1 score: 0.31506, speed: 0.23 step/s
eval loss: 0.14866, auc: 0.76428, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 4130, epoch: 3, batch: 1076, loss: 0.16074, auc: 0.74970, f1 score: 0.31503, speed: 0.04 step/s
global step 4140, epoch: 3, batch: 1086, loss: 0.14003, auc: 0.75421, f1 score: 0.29320, speed: 0.23 step/s
global step 4150, epoch: 3, batch: 1096, loss: 0.15105, auc: 0.74739, f1 score: 0.28937, speed: 0.24 step/s
global step 4160, epoch: 3, batch: 1106, loss: 0.16329, auc: 0.75016, f1 score: 0.27790, speed: 0.23 step/s
eval loss: 0.15029, auc: 0.76022, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 4170, epoch: 3, batch: 1116, loss: 0.14738, auc: 0.74143

## Load trained model

In [16]:
# Load the optimal parameters of the trained model.
model.set_dict(paddle.load('ernie_ckpt/model_state.pdparams'))

# Load the parameters of the previously trained model.
# model.set_dict(paddle.load('/home/aistudio/work/model_state.pdparams'))

# Model Validation.
print("ERNIE 3.0 performance on GoEmotions micro-emotion 28 classification test set：", end= " ")
results = evaluate(model, criterion, metric, test_data_loader, label_vocab)

ERNIE 3.0 performance on GoEmotions micro-emotion 28 classification test set： eval loss: 0.14870, auc: 0.76312, f1 score: 0.30410, precison: 0.32904, recall: 0.28267


# Use the model to make predictions about the sentiment contained in the sentences.

In [17]:
# Define data loading and processing functions.
from paddlenlp.data import JiebaTokenizer, Pad, Stack, Tuple, Vocab
def convert_example(example, tokenizer, max_seq_length=64, is_test=False):
    qtconcat = example["text"]
    encoded_inputs = tokenizer(text=qtconcat, max_seq_len=max_seq_length)
    input_ids = encoded_inputs["input_ids"]
    token_type_ids = encoded_inputs["token_type_ids"]
    if not is_test:
        label = np.array([example["label"]], dtype="int64")
        return input_ids, token_type_ids, label
    else:
        return input_ids, token_type_ids

# Define the model prediction function.
def predict(model, data, tokenizer, label_vocab, batch_size=1, max_seq=64):
    examples = []
    # Process input data (list format) into a format acceptable to the model.
    for text in data:
        input_ids, segment_ids = convert_example(
            text,
            tokenizer,
            max_seq_length=max_seq,
            is_test=True)
        examples.append((input_ids, segment_ids))

    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input id
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # segment id
    ): fn(samples)

    # Seperates data into some batches.
    batches = []
    one_batch = []
    for example in examples:
        one_batch.append(example)
        if len(one_batch) == batch_size:
            batches.append(one_batch)
            one_batch = []
    if one_batch:
        # The last batch whose size is less than the config batch_size setting.
        batches.append(one_batch)

    results = []
    model.eval()
    for batch in batches:
        input_ids, segment_ids = batchify_fn(batch)
        input_ids = paddle.to_tensor(input_ids)
        segment_ids = paddle.to_tensor(segment_ids)
        logits = model(input_ids, segment_ids)
        probs = F.sigmoid(logits)
        probs = probs.tolist()
        # The results were processed by selecting the sentiment categories with probability greater than 0.5.
        for prob in probs:
            result = []
            for c, pred in enumerate(prob):
                if pred > 0.5:
                    result.append(label_vocab[c])
            results.append(','.join(result))
    return results  # Return prediction results

In [18]:
# Define the text data to be subjected to micro-sentiment analysis.
data = [
    # 11 disgust
    {"text": 'Thats absolutely disgusting.'},
    # 7 curiosity
    {"text":'Why would I do that?'},
    # 2 anger
    {"text":"You shut your mouth"},
    # 15 gratitude
    {"text":"Thank you."}
]

# Model Predictions.
labels =  predict(model, data, tokenizer, label_vocab, batch_size=1)

# Output prediction results
for idx, text in enumerate(data):
    print('Text: {} \t Lables: {}'.format(text['text'], labels[idx]))

Text: Thats absolutely disgusting. 	 Lables: 
Text: Why would I do that? 	 Lables: 
Text: You shut your mouth 	 Lables: 
Text: Thank you. 	 Lables: 
